In [32]:
import requests as req
import lxml.html
import json
import re, os

In [33]:
def get_song_tags(artist = "", track = ""):
    tmp_artist = artist.lower().replace(' ', '+')
    tmp_track = track.lower().replace(' ', '+')
    url = f'https://www.last.fm/music/{tmp_artist}/_/{tmp_track}'
    html_from_last_fm = req.get(url).text
    html = lxml.html.fromstring(html_from_last_fm)
    track_dict = {'artist': artist, 'track': track, 'tags':[]}
    for li in html.xpath("//ul[@class='tags-list tags-list--global']/li"):
        track_dict['tags'].append(li.text_content())
    return track_dict

In [34]:
def get_track_list(path):
    for file in os.listdir(path):
        track_info = re.split('[-]', file[:-4])
        if track_info[0][-1] == ' ':
            track_info[0] = track_info[0][:-1]
        if track_info[1][0] == ' ':
            track_info[1] = track_info[1][1:]
        yield track_info[0], track_info[1], file

In [45]:
def get_json(path):
    info = []
    #TODO добавить проверку на наличие тегов. Если нет тегов дальше ничего не делаем
    for artist, track, file_name in get_track_list(path):
        meta = get_song_tags(artist= artist, track= track)
        meta['file_name'] = file_name
        info.append(meta)
    return info

In [46]:
def dump(info, path, user_id):
    with open(path+f'{user_id}_info.json', 'w') as fp:
        json.dump(info, fp)

In [49]:
info = get_json('.//tracks//')
print(info)
dump(info,'', 1)

[{'artist': 'Best Songs Of Elton John', 'track': 'Elton John Greatest Hits Playlist', 'tags': [], 'file_name': 'Best Songs Of Elton John - Elton John Greatest Hits Playlist.mp3'}, {'artist': 'Billy Joel, Elton John', 'track': 'Piano Man (live)', 'tags': ['classic rock'], 'file_name': 'Billy Joel, Elton John - Piano Man (live).mp3'}, {'artist': 'DJ Snake', 'track': 'Magenta Riddim', 'tags': ['party', '2018 single', 'moombahton'], 'file_name': 'DJ Snake - Magenta Riddim.mp3'}, {'artist': 'Elthon John', 'track': 'Sacrifice', 'tags': ['pop', '80s', 'elton john', 'classic rock', 'british'], 'file_name': 'Elthon John - Sacrifice.mp3'}, {'artist': 'Elton John', 'track': 'Bennie and the Jets (with lyrics)', 'tags': ['pop', 'classic rock', 'singer-songwriter', 'rock', 'piano'], 'file_name': 'Elton John - Bennie and the Jets (with lyrics).mp3'}, {'artist': 'Elton John', 'track': 'Can you feel the love tonight', 'tags': ['soundtrack', 'disney', 'pop', '90s', 'piano'], 'file_name': 'Elton John - C